### Some EDA!

In [411]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from  sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from pandas_confusion import ConfusionMatrix

In [412]:
df_raw = pd.read_json('data/train_new.json')

In [413]:
# count NaN's, add column
temporary_df = df_raw.replace(r'\s+( +\.)|#',np.nan,regex=True).replace('',np.nan)
missin_list=[]
for i, j in enumerate(temporary_df.count(1)):
    missin_list.append((temporary_df.shape[1])-j)
df_raw['missing_count'] = missin_list

In [414]:
# fill NaN's
df_raw.fillna(0,inplace=True)

In [415]:
# create labels
df_raw['label'] = np.where((df_raw['acct_type']=='fraudster_event') | (df_raw['acct_type']=='fraudster') , 1, 0)

In [416]:
# percentage of fraud
fraud = df_raw[df_raw.label==1]
not_fraud = df_raw[df_raw.label==0]
float(fraud.acct_type.count()) / not_fraud.acct_type.count()

0.09870488160012261

In [417]:
# create uppercase percentage metric
# rad maps!
df_raw['uprcse_ltrs'] = map(lambda message: sum(1 for c in message if c.isupper()), df_raw['name'])
df_raw['name_len'] = map(lambda message: len(message), df_raw['name'])
df_raw['uprcse_percent'] = df_raw['uprcse_ltrs'] / df_raw['name_len']
df_raw.uprcse_percent.fillna(np.mean(df_raw.uprcse_percent),inplace=True)

In [418]:
# create name_length columns
df_raw['no_name'] = np.where(df_raw['name_length']==0, 1, 0)

In [419]:
# location feature engineering
df_raw['event_duration'] = abs(df_raw['event_start'] - df_raw['event_end'])
dummies = pd.get_dummies(df_raw['venue_state'])
df_raw['loc'] = dummies.values.argmax(axis=1)

In [420]:
# build X and y df
df_x = df_raw[['venue_latitude','venue_longitude','no_name','uprcse_percent','gts','channels','user_type','user_age',
               'has_logo','body_length','name_length','org_twitter','org_facebook','event_created','event_published',
              'event_end','event_start','event_duration','loc','missing_count']]
y = df_raw[['label']]

In [432]:
# train / test split
X_train, X_test, y_train, y_test = train_test_split(df_x, y,test_size=0.2, random_state=42)

In [465]:
# build Random Forest
model = RandomForestClassifier(n_estimators=50,oob_score=True)
model.fit(X_train, y_train)
model.oob_score_

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.96512337605719767

In [466]:
preds = model.predict(X_test)
preds = pd.DataFrame(preds,columns = ['prediction'])
print preds.prediction.value_counts()

0    2682
1     186
Name: prediction, dtype: int64


In [467]:
model.score(X_test, y_test)

0.96861924686192469

In [468]:
confusion_matrix(y_test,preds)

array([[2603,   11],
       [  79,  175]])

In [408]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 52 columns):
acct_type             14337 non-null object
approx_payout_date    14337 non-null int64
body_length           14337 non-null int64
channels              14337 non-null int64
country               14337 non-null object
currency              14337 non-null object
delivery_method       14337 non-null float64
description           14337 non-null object
email_domain          14337 non-null object
event_created         14337 non-null int64
event_end             14337 non-null int64
event_published       14337 non-null float64
event_start           14337 non-null int64
fb_published          14337 non-null int64
gts                   14337 non-null float64
has_analytics         14337 non-null int64
has_header            14337 non-null float64
has_logo              14337 non-null int64
listed                14337 non-null object
name                  14337 non-null object
name_length     

In [426]:
df_x

,venue_latitude,venue_longitude,no_name,uprcse_percent,gts,channels,user_type,user_age,has_logo,body_length,name_length,org_twitter,org_facebook,event_created,event_published,event_end,event_start,event_duration,loc,missing_count
0,25.777471,-80.133433,0,0.783333,0.00,5,1,36,0,3852,60,0.0,0.0,1262739706,1.263110e+09,1265630400,1265594400,36000,127,3
1,32.776566,-79.930922,0,0.222222,868.02,0,3,149,1,3499,27,12.0,0.0,1293832670,1.293833e+09,1296288000,1296255600,32400,338,1
2,33.944201,-118.080419,0,0.214286,3500.00,8,3,214,0,2601,28,0.0,0.0,1291090956,1.291092e+09,1295740800,1295713800,27000,52,1
3,0.000000,0.000000,0,0.238095,1167.35,6,3,889,1,12347,21,0.0,0.0,1360681570,1.360683e+09,1388534400,1360702800,27831600,0,10
4,42.353848,-71.044276,0,0.106061,2313.15,11,3,35,0,2417,66,0.0,0.0,1291994666,1.291995e+09,1297468800,1297440000,28800,214,0
5,38.209797,-84.558831,0,0.090909,302.57,6,1,299,1,117,44,0.0,0.0,1294421810,1.294422e+09,1300064400,1300053600,10800,176,1
6,39.414270,-77.405089,0,0.083333,832.64,5,3,706,1,28,36,0.0,0.0,1294425018,1.294425e+09,1297477800,1297468800,9000,217,2
7,35.580468,-82.563855,0,0.142857,220.64,12,3,71,1,974,28,0.0,18.0,1294427837,1.294428e+09,1296277200,1296271800,5400,245,1
8,37.792847,-122.402082,0,0.157895,3706.36,4,3,0,1,4388,19,0.0,0.0,1294428122,1.294880e+09,1297753200,1297737000,16200,1,6
9,35.580468,-82.563855,0,0.142857,535.84,12,3,71,1,974,28,0.0,18.0,1294428286,1.294428e+09,1296882000,1296876600,5400,245,1
